# 0.0 IMPORTS

In [1]:
import pandas as pd

## 0.1 Helper Functions

In [29]:
def jupyter_settings(): 
    %matplotlib inline 
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [24, 20]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') ) 
    pd.options.display.max_columns = None
    pd.options.display.max_rows = 1000
    pd.set_option( 'display.expand_frame_repr', False ) 
    sns.set()

In [30]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


NameError: name 'HTML' is not defined

## 0.2 Loading Data

In [14]:
df_train_raw

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [21]:
# loading data
df_store_raw = pd.read_csv('../data/raw/store.csv', low_memory=False)
df_train_raw = pd.read_csv('../data/raw/train.csv', low_memory=False)
df_test_raw = pd.read_csv('../data/raw/test.csv', low_memory=False)

# merge
df_raw = pd.merge(df_train_raw, df_store_raw, on=['Store'], how='inner')

In [23]:
df_raw.sample(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
1009455,1107,4,2014-02-06,6461,661,1,1,0,0,a,a,1400.0,6.0,2012.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
811229,890,1,2013-10-14,3868,364,1,0,0,0,a,a,4450.0,NaN,NaN,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
771300,847,3,2015-03-25,5337,743,1,0,0,0,c,c,190.0,NaN,NaN,1,31.0,2013.0,"Feb,May,Aug,Nov"
711988,782,5,2014-01-03,4577,478,1,0,0,1,c,c,5390.0,8.0,2003.0,1,1.0,2012.0,"Jan,Apr,Jul,Oct"
19309,21,1,2013-04-15,3171,386,1,0,0,0,c,c,550.0,10.0,1999.0,1,45.0,2009.0,"Jan,Apr,Jul,Oct"


# 1.0 DATA DESCRIPTION

In [33]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [34]:
import inflection

cols_old = df1.columns

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map (snakecase, cols_old))

#rename
df1.columns = cols_new

# 2.0 FEATURE ENGINEERING

# 3.0 DATA FILTERING

# 4.0 DATA EXPLORATORY ANALYSIS

# 5.0 DATA PREPARATION

# 6.0 FEATURE SELECTION

# 7.0 MACHINE LEARNING MODELING

# 8.0 HYPER PARAMETERS FINE TUNING

# 9.0 TRADUTION AND INTERPRETATE ERROR

# 10.0 DEPLOY MODEL TO PRODUCTION